# Notebook Overview
This notebook, created by Lina Lopes in December 2024, demonstrates the use of a pre-trained machine learning model for real-time body segmentation (Media Pipe). The goal is to identify the silhouette of a body and transmit the output to a Syphon server, enabling dynamic masking for projection mapping in videomapping software.

The segmentation leverages MediaPipe's Selfie Segmentation model to process live video feeds, isolating the body region and creating a mask. This mask can be used creatively as a dynamic layer for visual projections, allowing for immersive and interactive installations.

For more projects, visit [Lina Lopes on GitHub](https://github.com/LinaLopes).

## Imports

Purpose: This cell imports the necessary libraries:

- cv2: OpenCV for video capture and manipulation.
- mediapipe: Google's framework for segmentation and other AI tasks.
- numpy: Numerical operations.
- Syphon: For video stream sharing.
- glfw: For managing windows and OpenGL context.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import Syphon
import glfw

## Initialization and Configuration

1. Initializes MediaPipe Selfie Segmentation for real-time body segmentation with model_selection=1 for high-quality results.
2. Defines the output window size for streaming: (1280, 720).
3. Sets up a Syphon server named "Syphon Body Silhouette" for transmitting video frames to external applications like MadMapper.

In [2]:
# Initialize MediaPipe Selfie Segmentation
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

# window details
size = (1280, 720)

# Configure o servidor Syphon
server = Syphon.Server("Syphon Body Silhouette", size, show=False)

I0000 00:00:1733336662.852597 2264590 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733336662.869466 2264942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-04 19:24:22.984 python[34492:2264590] SYPHON DEBUG: SyphonServerConnectionManager: Start Connection
2024-12-04 19:24:22.984 python[34492:2264590] SYPHON DEBUG: SyphonServerConnectionManager: Created connection with UUID: info.v002.Syphon.20C83AC0-9593-42B5-9D63-C0975634B762


## Segmentation Test/Example

1. Capture Input: Initializes the video capture (update the index 3 to match your device).
2. Mirroring (Optional): Flips the frame for a mirrored effect.
3. Preprocessing: Converts frames from BGR to RGB (MediaPipe's input requirement).
4. Segmentation Mask: Processes the frame with MediaPipe, generating a binary mask to isolate the body.
5. Blurring: Softens mask edges using Gaussian blur for smoother segmentation.
6. Body Isolation: Combines the mask and frame to display only the segmented body region.
7. Display: Shows the segmented output in a window titled "Body Segmentation".
8. Exit Condition: Stops processing when the 'q' key is pressed.

In [9]:
# Start video capture
cap = cv2.VideoCapture(3)  # Change 0 to the desired camera index

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    # Flip the frame horizontally for a mirrored view (optional)
    frame = cv2.flip(frame, 1)

    # Convert BGR frame to RGB as MediaPipe requires RGB input
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to get segmentation results
    results = selfie_segmentation.process(rgb_frame)

    # Create a mask where the detected body is white (255) and the background is black (0)
    body_mask = (results.segmentation_mask > 0.6).astype(np.uint8) * 255 # Replace 0.5 with a larger value, such as 0.6 or 0.7, to include only areas with higher confidence.

    # Use the cv2.GaussianBlur or cv2.blur filter to soften the edge. This softens the binary mask, creating a more gradual transition.
    body_mask = cv2.GaussianBlur(body_mask, (15, 15), 0)

    # Extract only the body region from the original frame
    body_only = cv2.bitwise_and(frame, frame, mask=body_mask)

    # Display the segmented body region
    cv2.imshow("Body Segmentation", body_only)

    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

## To send by Syphon to MadMapper

1. Segmentation Process - Confidence Threshold
    ```
    body_mask = (results.segmentation_mask > 0.6).astype(np.uint8) * 255
    ```

    The 0.6 threshold determines how confident the model must be to classify a pixel as part of the body. Higher thresholds (e.g., 0.7) make the segmentation more selective, reducing false positives but potentially missing some body parts. Lower thresholds (e.g., 0.5) make it more inclusive but may include background noise. Students can experiment with this value to balance segmentation quality.

2. Gaussian Blur
    ```
    body_mask = cv2.GaussianBlur(body_mask, (15, 15), 0)
    ```

    The Gaussian blur smooths the mask edges, reducing sharp transitions and artifacts. This is particularly useful when displaying the segmented region, making it visually pleasing and avoiding jagged edges. The kernel size (15, 15) can be adjusted to increase or decrease the level of blurring.

3. Syphon Integration
    ```
    server.publish_frame(body_only)
    ```

    This line sends the processed frame (segmented body) to the Syphon server. Tools like MadMapper can then receive and display this output in real time.



In [3]:
# Start video capture
cap = cv2.VideoCapture(3)  # Change 3 to the desired camera index

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    # Flip the frame horizontally for a mirrored view (optional)
    frame = cv2.flip(frame, 1)

    # Convert BGR frame to RGB as required by MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to get segmentation results
    results = selfie_segmentation.process(rgb_frame)

    # Create a mask where the detected body is white (255) and the background is black (0)
    body_mask = (results.segmentation_mask > 0.6).astype(np.uint8) * 255

    # Create a blank image (black background) and apply the mask as white
    silhouette = np.zeros_like(frame, dtype=np.uint8)  # Black background
    silhouette[body_mask == 255] = [255, 255, 255]  # White silhouette

    # Resize the silhouette to match the server size
    silhouette_resized = cv2.resize(silhouette, size)

    # Convert the silhouette to RGB format as required by Syphon
    silhouette_rgb = cv2.cvtColor(silhouette_resized, cv2.COLOR_BGR2RGB)

    # Send the silhouette to the Syphon server
    server.draw_and_send(silhouette_rgb)

    # Display the silhouette locally (optional)
    cv2.imshow("Body Silhouette", silhouette)

    # Exit the loop when 'q' is pressed
    key = cv2.waitKey(1) & 0xFF  # Read key press
    if key == ord('q'):  # Check if 'q' is pressed
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

2024-12-04 19:24:31.873 python[34492:2264590] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
